In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [14]:
#Assign the basic variables
#Random Task - Predict the position of a commuter based on some driving data
N = 10 #Number of samples (10 days of data)
T = 20 #Number of time series windows (20 sample points between commute)
D = 4 #Number of dimensions in data (Location & Speed - Lat,Long,Elev,Spd)
M = 4 #Number of hidden units 
K = 3 #Number of output units (Lat, Long, Elev)

In [16]:
X = np.random.randn(N, T, D) #Input
Y = np.random.randn(N, K) #Output
print(X[0])
print(Y[0])

[[ 0.3300333  -0.26859804  0.79939947  0.395754  ]
 [-0.49812999  0.85267715 -1.74605935 -1.32199702]
 [-0.40181503  0.00475293 -0.86383846 -2.04077906]
 [ 0.17229486  0.41660823 -0.23433197 -0.15784695]
 [-1.14085436  1.56128243  1.78066839  0.18273662]
 [-1.55605146 -0.08365172 -0.75660135  1.69493122]
 [ 0.05481932  0.24295508  0.43612059 -0.4055512 ]
 [-0.18096596  0.51600017  0.22943408 -1.33663844]
 [-0.28159546 -0.04483446 -0.20858437  0.54363225]
 [ 0.32945123 -0.65039997 -0.09187853 -1.42643032]
 [-1.59631915  0.94896876  0.65029419  1.97133962]
 [-0.69977343  0.63623289  1.09604828  0.13884948]
 [ 0.40300137 -1.49219248  0.43764684 -1.75912862]
 [ 0.94285736 -0.43206195 -0.41038023  0.5111171 ]
 [-0.0909263   0.78238205  1.51921658  0.12582324]
 [ 1.08646858 -0.07690655  0.46686822  1.53900304]
 [-0.10992708 -1.42571429 -1.25537803 -0.14616125]
 [ 0.4233613   0.59453185  1.09305327 -0.20565117]
 [-0.25883126  0.75436674  0.59993461  1.32311072]
 [ 0.90426799 -0.49124473  0.50

In [17]:
#Build model
input_layer = tf.keras.layers.Input(shape=(T, D))
hidden_layer = tf.keras.layers.SimpleRNN(M)(input_layer)
output_layer = tf.keras.layers.Dense(K)(hidden_layer)
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20, 4)]           0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 4)                 36        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [22]:
#compile and fit the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.1), loss=tf.keras.losses.MeanSquaredError())
model.fit(x=X, y=Y, epochs=60, verbose=0)

In [25]:
Y_pred = model.predict(X)
print(np.mean(tf.keras.losses.mse(Y[:,0], Y_pred[:,0])))  #Mean Error in Lat
print(np.mean(tf.keras.losses.mse(Y[:,1], Y_pred[:,1])))  #Mean Error in Long
print(np.mean(tf.keras.losses.mse(Y[:,2], Y_pred[:,2])))  #Mean Error in Elev

0.06835234
0.01047355
0.0737173


In [24]:
print(Y)
print(Y_pred)

[[ 0.37765407 -0.54538564 -2.1657667 ]
 [-0.96711081  0.22967925 -0.3425056 ]
 [-1.30759626  1.02372114 -0.07733552]
 [-0.60588849  1.24836915  0.87613897]
 [-1.02545158  1.28259371 -1.84136681]
 [-0.73413666 -0.71786171 -0.15044688]
 [-1.01049033 -1.11709124 -0.38416004]
 [ 1.59853722  0.12395179 -1.37085356]
 [-1.6602522   0.74857561 -2.10632467]
 [ 0.05044456 -1.29817285  0.07344798]]
[[ 0.35415947 -0.5498009  -2.20571   ]
 [-0.73393154  0.10061494  0.24715918]
 [-1.6896828   0.8119849  -0.33248606]
 [-0.6306201   1.2808334   0.68253857]
 [-1.2529776   1.2295997  -1.9173849 ]
 [-0.7040311  -0.6834549  -0.10698122]
 [-0.69594103 -0.922704   -0.16338396]
 [ 1.5940179   0.13574228 -1.3732263 ]
 [-1.2128453   0.7548388  -1.8822789 ]
 [-0.31037116 -1.283495   -0.34925503]]


In [29]:
#Lets look into the weights of the network
weights_rnn = model.get_weights()
len(weights_rnn)

5

In [30]:
#There are 5 weight/bias numpy arrays
#Array 0 - Wxh - Connections between Input and hidden units (4 x 4)
Wxh = weights_rnn[0]
#Array 1 - Whh - Recurring connection between the two states (t and t-1) of hidden units (4 x 4)
Whh = weights_rnn[1]
#Array 2 - Bh - Bias of hidden units 
Bh = weights_rnn[2]
#Array 3 - Who - Connections between hidden and output units (4 x 3)
Who = weights_rnn[3]
#Array 4 - Bo - Bias of output units
Bo = weights_rnn[4]

#We can also get the weights by layers
# layers[0] - Input layer - no weights
Wxh1, Whh1, Bh1 = model.layers[1].get_weights() #hidden layer
Who1, Bo1 = model.layers[2].get_weights() #output layer

#Just make sure these arrays are the same, so that we can use any of these methods in future
np.where(Wxh != Wxh1)
np.where(Who != Who1)


(array([], dtype=int64), array([], dtype=int64))

In [37]:
#Verify the operation that is being done by the RNN

Y_pred_manual = np.empty((N,K))

for i in range(N):
  x = X[i] #Compute each sample
  h_previous = np.zeros(M) #Initial h0 state is zeros
  y_pred = []  #keep track of each time sample output
  for t in range(T):
    h = np.tanh(x[t].dot(Wxh) + h_previous.dot(Whh) + Bh)
    y = h.dot(Who) + Bo
    h_previous = h
    y_pred.append(y)
  print("All Y predictions for each time instance in Sample{0}".format(i+1))
  print(y_pred)
  Y_pred_manual[i] = y_pred[-1] # we just need the last prediction


All Y predictions for each time instance in Sample1
[array([-1.8852619 ,  1.20111786, -0.86430777]), array([ 1.83530467, -0.3255989 , -1.14943237]), array([ 0.92316755,  2.64782663, -2.7003636 ]), array([ 0.08664711, -1.02288094, -0.56111675]), array([-0.78971326,  1.67212775,  0.69423653]), array([ 0.80704056,  0.0426585 , -2.35033914]), array([-0.61708387, -1.02450639, -0.16773155]), array([ 1.23481015, -0.19692565, -0.86032073]), array([ 0.7420477 ,  2.30311851, -2.53515334]), array([-1.15084647,  1.79350722, -1.94625763]), array([-0.97347074,  1.35270678,  0.69997589]), array([-0.56016642,  0.70389995,  0.55093305]), array([ 0.39545714, -0.71335269, -2.2562906 ]), array([-1.18559236,  1.50302449, -1.79078239]), array([-2.50519474,  0.53169154,  1.24012072]), array([-2.48521186,  1.05375914, -0.49770649]), array([-0.69353077, -0.70407127, -1.90333265]), array([-2.55599203,  0.53136257,  1.26509388]), array([-0.64096138,  1.16400985,  0.6573564 ]), array([ 0.35415975, -0.54980073, -2

In [42]:
#Check if manual RNN calculation and SimpleRNN is the same 
np.allclose(Y_pred, Y_pred_manual, atol=1e-03)

True